In [158]:
def Puerta(name):
    if name == "H":
        return 1/np.sqrt(2)*np.array([[1,1],[1,-1]],dtype=np.float128)
    if name == "swap":
        return np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
    if name == "cx":
        return np.array([[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]])
    
    raise ValueError("Puerta no definida")

In [199]:
def swaps(qubits):
    num_qubits = qubits[1] - qubits[0] + 1
    resta = -2**qubits[0]
    suma = 2**qubits[1]
    swap = np.eye(2**num_qubits)
    mask1 = 1 << qubits[0]
    #anti_mask = ((1 << num_qubits) - 1) ^ mask1
    mask2 = 1 << qubits[1]

    for i in range(2**num_qubits):
        if (i & mask1) and (i & mask2) == 0:
            swap[i,i] = 0
            swap[i+resta+suma,i+resta+suma] = 0
            swap[i,i+resta+suma] = 1
            swap[i+resta+suma,i] = 1

    return swap

In [200]:
swaps([0,2])

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [168]:
np.kron(Puerta("swap"),np.eye(2)) @ np.kron(np.eye(2),Puerta("swap")) @ np.kron(Puerta("swap"),np.eye(2))

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [171]:
num_qubits = 3
qubit = 0
mask = 1 << qubit
mask2 = 1 << 2

anti_mask = ((1 << num_qubits) - 1) ^ mask

print(mask)
print(anti_mask)
print(mask2)



1
6
4


In [112]:
import numpy as np

class QRegistry:
    def __init__(self,nqubits):
        self.nqubits = nqubits
        self.estado = np.append(np.array([1],dtype=complex),np.array([0 for _ in range(2**self.nqubits - 1)],dtype=complex)).reshape(-1,1)

    def ket(self):
        return self.estado
    
    def bra(self):
        return np.conjugate(np.transpose(self.estado))
    
    def M_densidad(self):
        return np.dot(self.ket(),self.bra())
    
    def aplicar_puerta(self, puerta, qubit):
        if len(qubit) == 1:
            if qubit[0] == 0:
                operacion = np.kron(np.eye(2**(self.nqubits - 1)),puerta)
            elif qubit[0] == self.nqubits-1:
                operacion = np.kron(puerta,np.eye(2**qubit[0]))
            else:
                operacion = np.kron(np.kron(np.eye(2**qubit[0]),puerta),np.eye(2**(self.nqubits - qubit[0] - 1)))
        else:
            if 0 in qubit:
                operacion = puerta
            else:
                operacion = np.eye(2)

            for i in range(1,self.nqubits):
                if i in qubit:
                    operacion = np.kron(operacion,puerta)
                else:
                    operacion = np.kron(operacion,np.eye(2))

        self.estado = np.dot(operacion,self.ket())
        #print(operacion)
        return self.estado
    
    def aplicar_puertaM(self, puerta, qubit):
        num_q_Puerta = int(np.log2(puerta.shape[0]))

        if qubit < 0 or qubit >= self.nqubits:
            raise ValueError("Qubit imposible")
        
        elif num_q_Puerta > self.nqubits - qubit:
            raise ValueError("La puerta no cabe")
        
        if qubit == 0:
            operacion = np.kron(np.eye(2**(self.nqubits - num_q_Puerta)),puerta)
        elif qubit + num_q_Puerta - 1 == self.nqubits-1:
            operacion = np.kron(puerta,np.eye(2**qubit))
        else:
            operacion = np.kron(np.kron(np.eye(2**qubit),puerta),np.eye(2**(self.nqubits - qubit - num_q_Puerta)))

        self.estado = np.dot(operacion,self.ket())
        #print(operacion)
        return self.estado
    
    def medir(self, qubit):
        if qubit < 0 or qubit >= self.nqubits:
            raise ValueError("Qubit imposible")
        
        lista = [j for j in range(2**self.nqubits) if j//2**(qubit)%2]
        # print(lista)
        r = np.random.rand()
        p = sum(np.absolute(self.estado[i,0])**2 for i in lista)

        if r < p:
            #print("Mide 1")
            listadg = [j for j in range(2**self.nqubits) if j//2**(qubit)%2 == 0]
            for i in listadg:
                self.estado[i,0] = 0
            self.estado = self.estado/np.sqrt(p)
        else:
            #print("Mide 0")
            for i in lista:
                self.estado[i,0] = 0
            self.estado = self.estado/np.sqrt(1-p)

        return int(r < p)


In [157]:
cx = Puerta("cx")
H = Puerta("H")

par_epr = QRegistry(2)
par_epr.aplicar_puertaM(H,0)

par_epr.aplicar_puertaM(cx,0)

display(par_epr.ket())

print(par_epr.medir(0))

print(par_epr.medir(1))

array([[0.70710678+0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.70710678+0.j]], dtype=complex256)

1
1


In [111]:
registro = QRegistry(3)
registro.aplicar_puertaM(np.kron(hadamard,hadamard),1)
registro.ket()

array([[0.5+0.j],
       [0. +0.j],
       [0.5+0.j],
       [0. +0.j],
       [0.5+0.j],
       [0. +0.j],
       [0.5+0.j],
       [0. +0.j]], dtype=complex256)

In [82]:
registro = QRegistry(3)
hadamard = 1/np.sqrt(2)*np.array([[1,1],[1,-1]],dtype=np.float128)
registro.aplicar_puerta(hadamard,[0])
registro.aplicar_puerta(hadamard,[1])
registro.aplicar_puerta(hadamard,[2])
registro.aplicar_puerta(hadamard,[0])
registro.aplicar_puerta(hadamard,[1])
registro.aplicar_puerta(hadamard,[2])

array([[1.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j]], dtype=complex256)

In [83]:
registro = QRegistry(3)
registro.medir(1)

0

In [84]:
ket0 = QRegistry(2)
ket0.ket()

array([[1.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j]])

In [85]:
ket0.bra()

array([[1.-0.j, 0.-0.j, 0.-0.j, 0.-0.j]])

In [86]:
ket0.M_densidad()

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [87]:
ket0.aplicar_puerta(hadamard,qubit=[1])

array([[0.70710678+0.j],
       [0.        +0.j],
       [0.70710678+0.j],
       [0.        +0.j]], dtype=complex256)

In [88]:
ket0.medir(1)

0

In [89]:
ket0.ket()

array([[1.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j]], dtype=complex256)

In [90]:
registro = QRegistry(3)
registro.aplicar_puerta(hadamard,[0,1,2])
registro.ket()

array([[0.35355339+0.j],
       [0.35355339+0.j],
       [0.35355339+0.j],
       [0.35355339+0.j],
       [0.35355339+0.j],
       [0.35355339+0.j],
       [0.35355339+0.j],
       [0.35355339+0.j]], dtype=complex256)

In [91]:
registro.ket().shape[0]

8

In [98]:
registro = QRegistry(3)
registro.aplicar_puertaM(hadamard,[1])
registro.ket()

array([[0.70710678+0.j],
       [0.        +0.j],
       [0.70710678+0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j]], dtype=complex256)